# Data Preparation

## Imports

In [972]:
import pandas as pd
import numpy as np

from pathlib import Path
from joblib import dump

### Data Import

In [973]:
source_path = Path('../data/raw/employees_zurich.csv')
df = pd.read_csv(source_path)

df.head()

,stichtag,direktion_code,direktion_name,amt_bereich_code,amt_bereich_name,indikator,wert,einheit,anmerkung_1,anmerkung_2
0,2020-12-31,1,Regierungsrat und Staatskanzlei,1000,Regierungsrat und Staatskanzlei,Frauen,27,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN
1,2020-12-31,1,Regierungsrat und Staatskanzlei,1000,Regierungsrat und Staatskanzlei,Maenner,43,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN
2,2020-12-31,2,Direktion der Justiz und des Innern,2201,Generalsekretariat,Frauen,41,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN
3,2020-12-31,2,Direktion der Justiz und des Innern,2201,Generalsekretariat,Maenner,44,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN
4,2020-12-31,2,Direktion der Justiz und des Innern,2204,Staatsanwaltschaft,Frauen,289,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN


In [974]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stichtag          768 non-null    object 
 1   direktion_code    768 non-null    int64  
 2   direktion_name    768 non-null    object 
 3   amt_bereich_code  768 non-null    int64  
 4   amt_bereich_name  768 non-null    object 
 5   indikator         768 non-null    object 
 6   wert              768 non-null    int64  
 7   einheit           768 non-null    object 
 8   anmerkung_1       768 non-null    object 
 9   anmerkung_2       0 non-null      float64
dtypes: float64(1), int64(3), object(6)
memory usage: 60.1+ KB


In [975]:
df.describe()

,direktion_code,amt_bereich_code,wert,anmerkung_2
count,768.000000,768.000000,768.000000,0.0
mean,4.416667,4689.958333,220.325521,NaN
std,2.300350,2345.055424,1086.928974,NaN
min,1.000000,1000.000000,0.000000,NaN
25%,2.000000,2239.250000,31.000000,NaN
50%,4.000000,4250.000000,51.000000,NaN
75%,7.000000,7125.000000,79.000000,NaN
max,8.000000,8800.000000,14507.000000,NaN


## Data cleanup

### Remove data about part time employees

In [976]:
df.anmerkung_1.unique()

array(['Anzahl Anstellungen ohne Uebriges Personal (Mitglieder von Behoerden, Lernende, Praktikanten)',
       'Vollzeit = Beschaeftigungsgrad ≥ 90%',
       'Teilzeit = Beschaeftigungsgrad < 90%'], dtype=object)

In [977]:
df = df[df.anmerkung_1 == 'Anzahl Anstellungen ohne Uebriges Personal (Mitglieder von Behoerden, Lernende, Praktikanten)']

### Simplify date

In [978]:
df.stichtag.unique()

array(['2020-12-31', '2021-12-31'], dtype=object)

In [979]:
df['jahr'] = pd.DatetimeIndex(df['stichtag']).year

df.head()

,stichtag,direktion_code,direktion_name,amt_bereich_code,amt_bereich_name,indikator,wert,einheit,anmerkung_1,anmerkung_2,jahr
0,2020-12-31,1,Regierungsrat und Staatskanzlei,1000,Regierungsrat und Staatskanzlei,Frauen,27,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN,2020
1,2020-12-31,1,Regierungsrat und Staatskanzlei,1000,Regierungsrat und Staatskanzlei,Maenner,43,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN,2020
2,2020-12-31,2,Direktion der Justiz und des Innern,2201,Generalsekretariat,Frauen,41,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN,2020
3,2020-12-31,2,Direktion der Justiz und des Innern,2201,Generalsekretariat,Maenner,44,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN,2020
4,2020-12-31,2,Direktion der Justiz und des Innern,2204,Staatsanwaltschaft,Frauen,289,Anstellungsverhaeltnisse [Anzahl],Anzahl Anstellungen ohne Uebriges Personal (Mi...,NaN,2020


### Delete redundant columns

In [980]:
df = df.drop(['direktion_code', 'amt_bereich_code', 'anmerkung_1', 'anmerkung_2', 'stichtag'], axis=1, errors='ignore')
df.head()

,direktion_name,amt_bereich_name,indikator,wert,einheit,jahr
0,Regierungsrat und Staatskanzlei,Regierungsrat und Staatskanzlei,Frauen,27,Anstellungsverhaeltnisse [Anzahl],2020
1,Regierungsrat und Staatskanzlei,Regierungsrat und Staatskanzlei,Maenner,43,Anstellungsverhaeltnisse [Anzahl],2020
2,Direktion der Justiz und des Innern,Generalsekretariat,Frauen,41,Anstellungsverhaeltnisse [Anzahl],2020
3,Direktion der Justiz und des Innern,Generalsekretariat,Maenner,44,Anstellungsverhaeltnisse [Anzahl],2020
4,Direktion der Justiz und des Innern,Staatsanwaltschaft,Frauen,289,Anstellungsverhaeltnisse [Anzahl],2020


### Separate absolute from relative numbers

In [981]:
df.einheit.unique()

array(['Anstellungsverhaeltnisse [Anzahl]', 'Prozent'], dtype=object)

In [982]:
df = df[df.einheit == 'Anstellungsverhaeltnisse [Anzahl]']
df = df.drop(columns=['einheit'], axis=1)
df.head()

,direktion_name,amt_bereich_name,indikator,wert,jahr
0,Regierungsrat und Staatskanzlei,Regierungsrat und Staatskanzlei,Frauen,27,2020
1,Regierungsrat und Staatskanzlei,Regierungsrat und Staatskanzlei,Maenner,43,2020
2,Direktion der Justiz und des Innern,Generalsekretariat,Frauen,41,2020
3,Direktion der Justiz und des Innern,Generalsekretariat,Maenner,44,2020
4,Direktion der Justiz und des Innern,Staatsanwaltschaft,Frauen,289,2020


### Define unique identifier

In [983]:
df['identifier'] = df['direktion_name'] + ' ' + df['amt_bereich_name']

## Generate Additional Data for 1980 - 2020

In [984]:
length = int(len(df.jahr) / 2)
for year in reversed(range(1981, 2021)):
    np.random.seed(4)
    change = np.random.normal(-0.1, 0.062, length).reshape(-1, 1)
    df_year = df[df.jahr == year].copy()
    vals = np.array(df_year.wert).reshape(-1, 1)
    df_year.wert = np.floor(vals + change * vals)
    df_year.jahr = [year-1] * length
    df = pd.concat([df, df_year])

df = df.sort_values(['jahr', 'direktion_name', 'amt_bereich_name'], ascending=True)
df.head()

,direktion_name,amt_bereich_name,indikator,wert,jahr,identifier
90,Baudirektion,"Amt fuer Abfall, Wasser, Energie und Luft",Frauen,0.0,1980,"Baudirektion Amt fuer Abfall, Wasser, Energie ..."
91,Baudirektion,"Amt fuer Abfall, Wasser, Energie und Luft",Maenner,0.0,1980,"Baudirektion Amt fuer Abfall, Wasser, Energie ..."
94,Baudirektion,Amt fuer Landschaft und Natur,Frauen,41.0,1980,Baudirektion Amt fuer Landschaft und Natur
95,Baudirektion,Amt fuer Landschaft und Natur,Maenner,10.0,1980,Baudirektion Amt fuer Landschaft und Natur
86,Baudirektion,Amt fuer Raumentwicklung,Frauen,0.0,1980,Baudirektion Amt fuer Raumentwicklung


In [985]:
df.jahr = df.jahr.astype(int)
df.head()

,direktion_name,amt_bereich_name,indikator,wert,jahr,identifier
90,Baudirektion,"Amt fuer Abfall, Wasser, Energie und Luft",Frauen,0.0,1980,"Baudirektion Amt fuer Abfall, Wasser, Energie ..."
91,Baudirektion,"Amt fuer Abfall, Wasser, Energie und Luft",Maenner,0.0,1980,"Baudirektion Amt fuer Abfall, Wasser, Energie ..."
94,Baudirektion,Amt fuer Landschaft und Natur,Frauen,41.0,1980,Baudirektion Amt fuer Landschaft und Natur
95,Baudirektion,Amt fuer Landschaft und Natur,Maenner,10.0,1980,Baudirektion Amt fuer Landschaft und Natur
86,Baudirektion,Amt fuer Raumentwicklung,Frauen,0.0,1980,Baudirektion Amt fuer Raumentwicklung


## Save final dataframes

In [986]:
target_path = Path('../data/clean/employees.dump')
dump(df, target_path)

['../data/clean/employees.dump']